In [1]:
import findspark
findspark.init('c:/spark')
from pyspark import SparkConf, SparkContext
import collections
import re
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pandas as pd
import collections
from pyspark.ml.linalg import Vectors

In [2]:
spark = SparkSession.builder.config("spark.sql.warehouse.dir", "C:/temp").appName("SparkSQL").getOrCreate()

In [3]:
inputLines = spark.sparkContext.textFile("train_full_parsed_clean5.csv")

In [4]:
header=inputLines.filter(lambda l: "_id" in l)

In [ ]:
header.collect()

In [5]:
linesnoheader = inputLines.subtract(header)

In [6]:
topcoord=pd.read_csv("coordinate_more_than_1000.csv")

In [7]:
coordinates_name=topcoord['coordinates'].tolist()
coordinates_name2=topcoord['valname'].tolist()

In [8]:
def topcoordinates(line):
    coordinates=re.findall('(\[\-\d+\.\d+\,\s\d+\.\d+\])',line)
    row_coord=[1 if i in coordinates else 0 for i in coordinates_name ]
    feature=line.split(',')
    feature1=[float(i) for i in feature[:56]]
    feature2=[int(i) for i in feature1[8:56]]
    feature3=feature1[:8]
    return feature3+feature2+row_coord

    

In [9]:
coord = linesnoheader.map(topcoordinates)

In [10]:
feature=['log_duration', 'log_trip_duration', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'great_circle_distance','distance',
       'snow', 'holiday', 'vendor_id', 'pickup_hour_0',
       'pickup_hour_1', 'pickup_hour_2', 'pickup_hour_3', 'pickup_hour_4',
       'pickup_hour_5', 'pickup_hour_6', 'pickup_hour_7', 'pickup_hour_8',
       'pickup_hour_9', 'pickup_hour_10', 'pickup_hour_11',
       'pickup_hour_12', 'pickup_hour_13', 'pickup_hour_14',
       'pickup_hour_15', 'pickup_hour_16', 'pickup_hour_17',
       'pickup_hour_18', 'pickup_hour_19', 'pickup_hour_20',
       'pickup_hour_21', 'pickup_hour_22', 'pickup_hour_23',
       'pickup_weekday_0', 'pickup_weekday_1', 'pickup_weekday_2',
       'pickup_weekday_3', 'pickup_weekday_4', 'pickup_weekday_5',
       'pickup_weekday_6', 'pickup_month_1', 'pickup_month_2',
       'pickup_month_3', 'pickup_month_4', 'pickup_month_5',
       'pickup_month_6', 'passenger_count_0', 'passenger_count_1',
       'passenger_count_2', 'passenger_count_3', 'passenger_count_4',
       'passenger_count_5', 'passenger_count_6', 'passenger_count_7']
colNames=feature+coordinates_name2

In [11]:
train = coord.toDF(colNames)

In [ ]:
train.write.csv("train_full_parsed_clean5.csv",header = 'true')

In [ ]:
train.repartition(1).write.format('com.databricks.spark.csv').save('path+my.csv',header = 'true')

In [12]:
train.repartition(1).write.parquet('newdata6.parquet')

Py4JJavaError: An error occurred while calling o86.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$$anonfun$write$1.apply$mcV$sp(FileFormatWriter.scala:213)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$$anonfun$write$1.apply(FileFormatWriter.scala:166)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$$anonfun$write$1.apply(FileFormatWriter.scala:166)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:65)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:166)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:145)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:58)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:56)
	a

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import OneHotEncoder

In [ ]:
feature_col=colNames[0:1]+colNames[2:]
vecAssembler = VectorAssembler(inputCols=feature_col, outputCol="features")


In [ ]:
(trainingData, testData) = train.randomSplit([0.8, 0.2],seed=42)

In [ ]:

gbt = GBTRegressor(featuresCol="features",labelCol="log_trip_duration",maxIter=10,maxDepth=10, seed=42)

# Chain indexer and GBT in a Pipeline
pipeline = Pipeline(stages=[vecAssembler,gbt])

# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)

In [ ]:
from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
from pyspark.mllib.util import MLUtils

model = GradientBoostedTrees.trainRegressor(trainingData,
                                             categoricalFeaturesInfo={}, numIterations=3)